# Dataton BC 2018

**Grupo:** The kernel trick

**Fecha:** 29-10-2018

# Script # 3: Modelo de clasificación para asignar categorías a cada transacción

En este script se desarrolla un modelo de clasificación multi-clase para asignar alguna de las categorías encontradas en el script 2 a cada transacción, basándose en datos de la transacción y del pagador.
_____________________________________________________________________________________________________________________________


In [1]:
# Importar librerías

import numpy as np
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from xgboost import XGBClassifier

pd.options.mode.chained_assignment = None

semilla = 13

C:\Users\edescobar\AppData\Local\Continuum\miniconda3\envs\anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## 1. Cargar base de modelamiento y formar grupos de entrenamiento y prueba

In [2]:
# Cargar la tabla de modelamiento, tal y como salió del script # 2
data = pd.read_csv('base_modelo.csv')
data.head()

,id_cliente,id_trn_ach,valor_trx,dia_trx,mes_trx,dia_del_mes_trx,hora_trx,cat_tema,seg_str,ocupacion,tipo_vivienda,nivel_academico,estado_civil,genero,edad,ingreso_rango
0,116890,326250526,3120.20,Monday,April,2,21,Pago de impuestos,PERSONAL PLUS,1,NaN,S,I,F,32.0,f. (5.5 6.6MM]
1,162625,326254647,118031.07,Monday,April,2,21,Pago de impuestos,PERSONAL PLUS,E,F,U,S,M,37.0,h. (7.6 8.7MM]
2,104683,248482707,17711.09,Wednesday,March,22,11,Pago de impuestos,PERSONAL,1,F,U,M,F,43.0,b. (1.1 2.2MM]
3,189406,286116816,4957.62,Wednesday,October,4,14,Pago de impuestos,EMPRENDEDOR,NaN,NaN,NaN,NaN,NaN,NaN,i. (8.7 Inf)
4,62915,334420397,313244.24,Thursday,May,3,20,Pago de impuestos,PERSONAL PLUS,5,O,U,M,F,62.0,i. (8.7 Inf)


In [3]:
# Definición de columnas numéricas, categóricas y la variable objetivo

cols_num = ['valor_trx','dia_del_mes_trx','hora_trx','edad']
cols_cat = ['dia_trx','mes_trx','seg_str','ocupacion','tipo_vivienda','nivel_academico','estado_civil','genero',
           'ingreso_rango']
col_target = ['cat_tema']

X = data[cols_num + cols_cat]
y = data[col_target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=semilla)


In [4]:
# Comprobar que las clases de la variable objetivo estén distribuidas de manera similar en los grupos de entrenamiento y prueba

print('------ Distribución de datos en set de entrenamiento -----')
print(y_train.cat_tema.value_counts()/len(y_train))

print('\n------ Distribución de datos en set de entrenamiento -----')
print(y_test.cat_tema.value_counts()/len(y_test))

------ Distribución de datos en set de entrenamiento -----
Seguridad social               0.085749
Transporte/construcción        0.085379
Pago de impuestos              0.085356
Servicios públicos             0.085351
Tecnología y comunicaciones    0.085276
Trámites/gobierno              0.085206
Tarjetas de crédito            0.085160
Educación                      0.085149
Pago de deudas                 0.085133
Ahorro y giros                 0.085006
Vivienda                       0.084865
Otros                          0.062370
Name: cat_tema, dtype: float64

------ Distribución de datos en set de entrenamiento -----
Vivienda                       0.086438
Ahorro y giros                 0.086015
Pago de deudas                 0.085633
Educación                      0.085585
Tarjetas de crédito            0.085551
Trámites/gobierno              0.085415
Tecnología y comunicaciones    0.085203
Servicios públicos             0.084978
Pago de impuestos              0.084965
Transporte

## 2. Tratamiento de las variables para poder modelar

### 2.1 Tratamiento de variables numéricas

#### 2.1.1 Rellenar los datos numéricos faltantes con el promedio de cada columna

In [5]:
means = round(X_train[cols_num].mean())

X_train[cols_num] = X_train[cols_num].fillna(dict(means))
X_test[cols_num] = X_test[cols_num].fillna(dict(means))


#### 2.1.2 Normalizar las variables numéricas paraque tengan promedio= 0 y desv. estándar = 1

Esto puede ser de ayuda para el desempeño de los modelos clasificadores

In [6]:
scaler = StandardScaler()
X_train[cols_num] = scaler.fit_transform(X_train[cols_num])

# Se aplica al dataframe 'holdout'
X_test[cols_num] = scaler.transform(X_test[cols_num])


## 2.2 Tratamiento de las variables categóricas

Se hace *one hot encoding* (OHE) para las variables categóricas, para que puedan entrar al modelo.

In [7]:
X_train    = pd.get_dummies(X_train,drop_first=True,columns=cols_cat,dummy_na=True)
X_test = pd.get_dummies(X_test,drop_first=True,columns=cols_cat,dummy_na=True)


## 2.3 Reducción de dimensionalidad

Algunas de las variables categóricas tienen muchos valores posibles. Esto implica que después de aplicar el OHE, el dataset puede quedar con muchas variables. Para reducir el número de variables que entran al modelo, se hace análisis de componentes principales (PCA).

In [8]:
print('Cantidad de variables para modelar: {}\n'.format(len(X_train.columns)))

# Se ajusta el modelo PCA
pca = PCA(n_components=35)
Xp = pca.fit_transform(X_train)
Xp_test = pca.transform(X_test)

explained_var = pca.explained_variance_ratio_.cumsum()[-1]
print('Al reducir el dataset a 35 variables, aún se puede explicar el {} de la varianza de los datos.'.format(explained_var))


Cantidad de variables para modelar: 74

Al reducir el dataset a 35 variables, aún se puede explicar el 0.924649667829093 de la varianza de los datos.


## 3. Desarrollo del modelo

### 3.1 Entrenamiento del modelo

Se entrena un modelo de *Random Forest* con las bases una vez salen del PCA. Los parámetros elegidos para el modelo salen de una breve búsqueda. Por cuestiones de tiempo no se probaron más modelos, o se hizo una *grid search* más completa.

In [9]:
# Entrenamiento de modelo de Random Forest

rf = RandomForestClassifier(criterion='gini', max_depth=7, max_features='sqrt', n_estimators=200,random_state=semilla)
rf.fit(Xp,y_train)


C:\Users\edescobar\AppData\Local\Continuum\miniconda3\envs\anaconda\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=13, verbose=0, warm_start=False)

### 3.2 Validación del modelo

Primero se define una función para calcular y mostrar resultados sobre sets de entrenamiento y validación para un determinado modelo.

Luego se aplica esta función al modelo entrenado para ver su desempeño.

In [10]:
#  Se define una función para mostrar resultados sobre sets de entrenamiento y validación
def show_results(model,X_train, X_test, y_train, y_test):
    pred_train = model.predict(X_train)
    pred_test = model.predict(X_test)

    print('Resultados en set de entrenamiento')
    print('Accuracy: {}'.format(accuracy_score(y_train,pred_train)))
    print('Confusion matrix:')
    print(confusion_matrix(y_train,pred_train))

    print('\n--------------------------')

    print('Resultados en set de prueba')
    print('Accuracy: {}'.format(accuracy_score(y_test,pred_test)))
    print('Confusion matrix:')
    print(confusion_matrix(y_test,pred_test))

In [11]:
# Resultados del modelo de random forest
show_results(rf,Xp,Xp_test,y_train,y_test)

Resultados en set de entrenamiento
Accuracy: 0.2165850841554107
Confusion matrix:
[[ 5589  1284    16  3393  3753  5142  2242   545  7523   644  5698  1560]
 [ 2537  5226    19  4541  5190  4226  2032  1330  4162   538  6030  1621]
 [ 1275  1694   786  4339  3409  3342  1580  1068  3143   475  3736  2586]
 [ 1189  1736    21 10011  5296  3459  2363  1299  4510   627  4292  2642]
 [ 1273  1320     9  3563 14134  4077  1607   753  4050   248  5467  1042]
 [ 1763   947     5  1683  2943 15051  2221   251  6415   335  5692   410]
 [ 2007  1234    18  2093  4090  7590  4201   381  7798   587  7135   407]
 [ 1351  1793    15  7151  4787  4138  2453  2600  4994   633  5516  2026]
 [ 2625   822     7   384  2812  6998  2552    56 12167   518  8496    71]
 [ 1574  1970    17  6221  5038  4002  2417  1383  4447  1135  6685  2664]
 [ 1011   792    10   140  2781  4562  1433   135  6240   274 20074    25]
 [ 1783  1833    22  6857  5144  3555  2307  1103  4488   656  5290  4289]]

----------------

### 3.3 Análisis de resultados

Los resultados tienen un gran márgen de mejora, que puede ser alcanzado con una sintonización de parámetros más rigurosa utilizando validación cruzada, y considerando más modelos/ diferentes características.

Sin embargo, el modelo presentado en este script es un buen punto de partida. En primer lugar, el desempeño en los datos de prueba y en los de entrenamiento es muy similar, por lo que se puede concluir que no hay overfitting.

En segundo lugar, es importante poner en perspectiva el **21.2%** de *accuracy* obtenido en el conjunto de prueba. Al existir 12 diferentes categorías en las que puede clasificar una transacción, el porcentaje de acierto esperado de un modelo que clasifique de manera aleatoria es del 1/12 = **8.3%**. Partiendo de ese caso base, nuestro clasificador obtiene un desempeño 2.5 veces superior, lo que muestra el potencial de esta propuesta.